In [2]:
import json
import os
base_annotation_file_path = '/ebs/.cache/ubuntu/sharegpt_data/comparisons_gpt4/mistralsft0.11_vs_gpt4/annotations.json'
with open(base_annotation_file_path, 'r') as f:
    annotations = json.load(f)

In [3]:
new_sample_folder = '/ebs/.cache/ubuntu/sharegpt4_mistral7b_sft46k_lr5e-7_2023-11-28_09-43-28_375063/epoch-2/sharegpt2turn_noeos_maxlen512_temp1.0'
sft_instruction_truncoutput_pair = []
sft_instructions = []
instructions_too_long = 0

for sample_file in os.listdir(new_sample_folder):
    sft_outputs = json.load(open(os.path.join(new_sample_folder, sample_file), 'r'))
    for instruction, sft_output in sft_outputs.items():
        instruction_trimmed = instruction[len('Human: '):-len('\n\nAssistant: ')]
        if instruction_trimmed in sft_instructions:
            continue
        # really long instructions exceed the context length for GPT-4
        if len(instruction_trimmed.split()) >= 2000 or len(instruction_trimmed) >= 8000:
            instructions_too_long += 1
            continue
        sft_instruction_truncoutput_pair.append({'instruction': instruction_trimmed,
                                                 'output': sft_output[0]})
        sft_instructions.append(instruction_trimmed)

In [8]:
annotations[0]

{'instruction': 'I need to interview someone with 5 years experience in software development for a software engineering contractor job. The job will largely entail working with Spring Boot applications that use custom spring boot starter libraries, REST microservices, Kafka through the use of Spring Cloud Stream, OpenFeign, and we use Gradle to build our software. Can you help me build some questions appropriate for their number of years of experience?',
 'output_1': 'Certainly! Here are some questions you can ask during the interview to gauge their experience and knowledge in software development, particularly in the context of the technologies you mentioned:\n\n1. Can you tell us about your overall experience with software development and how it has evolved over the past 5 years?\n\n2. What kind of projects have you worked on using Spring Boot? Can you share some examples of custom spring boot starter libraries you have created or worked with?\n\n3. How do you ensure that your RESTfu

In [9]:
indices_to_remove = []
for idx, entry in enumerate(annotations):
    if entry['instruction'] in sft_instructions:
        entry['output_2'] = sft_instruction_truncoutput_pair[sft_instructions.index(entry['instruction'])]['output']
        entry['preference'] = 1
    else:
        indices_to_remove.append(idx)
print(len(indices_to_remove))
for idx in indices_to_remove:
    del annotations[idx]

0


In [12]:
# dump new annotations to file
new_annotation_file = os.path.join(new_sample_folder, 'auto_annotations_vs_gpt4.json')
with open(new_annotation_file, 'w') as f:
    json.dump(annotations, f, indent=4)